In [1]:
%load_ext autoreload
%autoreload 2
import theano
import theano.tensor as T
import numpy as np
import FSRCNN_Theano
import os
import Fpreprocessing
from scipy import ndimage,misc
from PIL import Image


Using gpu device 0: Tesla M2050 (CNMeM is disabled, cuDNN not available)
/mnt/anaconda2/lib/python2.7/site-packages/theano/tensor/signal/downsample.py:6: UserWarning: downsample module has been moved to the theano.tensor.signal.pool module.
  "downsample module has been moved to the theano.tensor.signal.pool module.")


In [2]:
def get_image_prefix(image_name):
    return image_name.split('_', 1)[0]
def get_image_width(image_name):
    yo = image_name.split("_")
    return yo[2]
def get_image_height(image_name):
    yo = image_name.split("_")
    return yo[3].split('.',1)[0]
def create_image(image_folder, output_folder, output_type = 'YCbCr',upsampling_factor = 4):
    reconstruct = dict()
    for (dirpath,dirnames,filenames) in os.walk(image_folder):
        print image_folder
        filenames.sort()
        #print(filenames)
        
        for counter,image_filename in enumerate(filenames):
            print(get_image_prefix(image_filename))
            print('\n')
            actual_image_name = get_image_prefix(image_filename)
            if(actual_image_name not in reconstruct):
                reconstruct[actual_image_name] = np.zeros((8,8,3))
            if image_filename.split('.')[-1] == 'bmp' and image_filename[0] != '.':
                if counter % 10 == 0:
                    print "processed:" + str(counter)
                image = misc.imread(os.path.join(image_folder,image_filename),flatten=False, mode = output_type)
                #(width,height,channel_depth)
                w = int(get_image_width(image_filename))
                h = int(get_image_height(image_filename))
                print (w,h)
                print(image.shape)
                print(reconstruct[actual_image_name][14*w:14*w+33,14*h:14*h+33,:].shape)
                reconstruct[actual_image_name][14*w:14*w+33,14*h:14*h+33,:] = image
                #misc.imshow(reconstruct[actual_image_name])
                #img = Image.fromarray(data, 'RGB')
                img = Image.fromarray(reconstruct[actual_image_name], 'RGB')
                img.save('my.png')
                img.show()
                misc.imsave(os.path.join(actual_image_name+'.bmp'),reconstruct[actual_image_name])
    #for key in recstruct:
        #misc.imsave(os.path.join(output_folder,actual_image_name+'.bmp'),reconstruct[actual_image_name])
#create_image('/home/ubuntu/Data/Validation_Subsamples_RGB_4','/home/ubuntu/Data/Reconstructed',output_type ='RGB',upsampling_factor = 4)

In [3]:
#Create Validation_Subsamples_RGB_4 and Validation_Subsamples_RGB_4_GT folders

Fpreprocessing.create_subimages('/home/ubuntu/Data/Set5',
'/home/ubuntu/Data/Validation_Subsamples_RGB_4/',
output_type ='RGB',upsampling_factor = 4)

Fpreprocessing.create_subimages('/home/ubuntu/Data/Training_Full',
'/home/ubuntu/Data/Training_Subsamples_RGB_4/',
output_type ='RGB',upsampling_factor = 4)



/home/ubuntu/Data/Set5
processed:0
/home/ubuntu/Data/Validation_Subsamples_RGB_4/
('\n', '/home/ubuntu/Data/Validation_Subsamples_RGB_4/mapping.py')
/home/ubuntu/Data/Training_Subsamples_RGB_4/
('\n', '/home/ubuntu/Data/Training_Subsamples_RGB_4/mapping.py')


In [4]:

Fpreprocessing.create_subimages('/home/ubuntu/Data/Set14',
'/home/ubuntu/Data/Test_Subsamples_RGB_4/',
output_type ='RGB',upsampling_factor = 4)

/home/ubuntu/Data/Set14
processed:0
processed:10
/home/ubuntu/Data/Test_Subsamples_RGB_4/
('\n', '/home/ubuntu/Data/Test_Subsamples_RGB_4/mapping.py')


In [2]:
import FSRCNN_Theano

#load dataset

#load training
data_x = FSRCNN_Theano.load_dataset('/home/ubuntu/Data/Training_Subsamples_RGB_4','data_x')
data_y = FSRCNN_Theano.load_dataset('/home/ubuntu/Data/Training_Subsamples_RGB_4_gt','data_y')
valid_x = FSRCNN_Theano.load_dataset('/home/ubuntu/Data/Validation_Subsamples_RGB_4','data_x')
valid_y = FSRCNN_Theano.load_dataset('/home/ubuntu/Data/Validation_Subsamples_RGB_4_gt','data_y')
test_x = FSRCNN_Theano.load_dataset('/home/ubuntu/Data/Test_Subsamples_RGB_4','data_x')
test_y = FSRCNN_Theano.load_dataset('/home/ubuntu/Data/Test_Subsamples_RGB_4_gt','data_y')
print "done loading\n\n"
print "data_x: " + str(data_x.shape)
print "data_y: " + str(data_y.shape)
print "valid_x: " + str(valid_x.shape)
print "valid_y: " + str(valid_y.shape)
print "test_x: " + str(test_x.shape)
print "test_y: " + str(test_y.shape)

/home/ubuntu/Data/Training_Subsamples_RGB_4
loading from npz
/home/ubuntu/Data/Training_Subsamples_RGB_4_gt
loading from npz
/home/ubuntu/Data/Validation_Subsamples_RGB_4
loading from npz
/home/ubuntu/Data/Validation_Subsamples_RGB_4_gt
loading from npz
/home/ubuntu/Data/Test_Subsamples_RGB_4
loading from npz
/home/ubuntu/Data/Test_Subsamples_RGB_4_gt
loading from npz
done loading


data_x: (22092, 3, 8, 8)
data_y: (22092, 3, 33, 33)
valid_x: (2488, 3, 8, 8)
valid_y: (2488, 3, 33, 33)
test_x: (14851, 3, 8, 8)
test_y: (14851, 3, 33, 33)


In [3]:
#Bicubic interp to save computation during training
upsampled_x = data_x #Fpreprocessing.upsample(data_x) #33,33,3 input images expected
up_val_x = valid_x #Fpreprocessing.upsample(valid_x) #33,33,3 input images expected
up_test_x = test_x #Fpreprocessing.upsample(test_x) #33,33,3 input images expected

#Reshape for training,valid,test

print upsampled_x.shape
print data_y.shape
print up_val_x.shape
print up_test_x.shape

upsampled_x = upsampled_x.reshape((22092,8*8*3))
data_y = data_y.reshape((22092,33*33*3))
up_val_x = up_val_x.reshape((2488,8*8*3))
valid_y = valid_y.reshape((2488,33*33*3))
up_test_x = up_test_x.reshape((14851,8*8*3))
test_y = test_y.reshape((14851,33*33*3))

print upsampled_x.shape
print data_y.shape
print up_val_x.shape
print up_test_x.shape

(22092, 3, 8, 8)
(22092, 3, 33, 33)
(2488, 3, 8, 8)
(14851, 3, 8, 8)
(22092, 192)
(22092, 3267)
(2488, 192)
(14851, 192)


In [7]:
shared_x = theano.shared(np.asarray(upsampled_x,
                                       dtype=theano.config.floatX),
                         borrow=True)
shared_y = theano.shared(np.asarray(data_y,
                                       dtype=theano.config.floatX),
                         borrow=True)
shared_val_x = theano.shared(np.asarray(up_val_x,
                                       dtype=theano.config.floatX),
                         borrow=True)
shared_val_y = theano.shared(np.asarray(valid_y,
                                       dtype=theano.config.floatX),
                         borrow=True)
shared_test_x = theano.shared(np.asarray(up_test_x,
                                       dtype=theano.config.floatX),
                         borrow=True)
shared_test_y = theano.shared(np.asarray(test_y,
                                       dtype=theano.config.floatX),
                         borrow=True)
#shared_y = T.cast(shared_y,'int32')

batch_size = 100
n_epochs = 100
lrs = [.001]
for lr in lrs:
    print "\n\n ****************************** lr = " + str(lr) +"******************************************"
    learning_rate = lr

    n_train_batches = upsampled_x.shape[0]/batch_size
    n_valid_batches = up_val_x.shape[0]/batch_size
    n_test_batches = up_test_x.shape[0]/batch_size



    val_model,test_model = FSRCNN_Theano.train_FSRCNN(shared_x,shared_y,
                             shared_val_x,shared_val_y,
                             shared_test_x,shared_test_y,
                            n_train_batches, n_valid_batches, n_test_batches, 
                             n_epochs, batch_size,learning_rate,upsampling_factor=4)



 ****************************** lr = 0.001******************************************
lovely....

filter_shape...: (64, 3, 9, 9)
image_shape...: (100, 3, 8, 8)
input shape....: Shape.0
lovely....

filter_shape...: (32, 64, 5, 5)
image_shape...: (100, 64, 8, 8)
input shape....: Shape.0
decon layer....

filter_shape...: (3, 32, 10, 10)
image_shape...: (100, 32, 8, 8)
input shape....: Shape.0
epoch 1, minibatch 220/220, validation cost 15588176.000000 mse/pixel: 17979.439453 pnsr: 6.396891
     epoch 1, minibatch 220/220, test cost of best model 15194572.000000 perpixel mse 17525.458984 and test pnsr 6.299674
epoch 2, minibatch 220/220, validation cost 15514904.000000 mse/pixel: 17894.929688 pnsr: 6.420382
     epoch 2, minibatch 220/220, test cost of best model 15112749.000000 perpixel mse 17431.082031 and test pnsr 6.326683
epoch 3, minibatch 220/220, validation cost 15370421.000000 mse/pixel: 17728.283203 pnsr: 6.461264
     epoch 3, minibatch 220/220, test cost of best model 14981014

In [11]:
reconstructed_imgs = np.zeros(((n_valid_batches+1)*batch_size, 3, 17, 17))
for i in xrange(n_valid_batches):
     cost,MSE_per_pixel,psnr,reconstucted_patches = val_model(i)
     reconstructed_imgs[i*batch_size:(i+1)*batch_size,:,:,:] = reconstucted_patches

FSRCNN_Theano.rebuild_images(reconstructed_imgs,'/home/ubuntu/Data/Validation_Subsamples_RGB_4/',patch_dim=17,dataset='validate')
reconstructed_imgs = np.zeros((14851, 3, 17, 17))
for i in xrange(n_test_batches):
     cost,MSE_per_pixel,psnr,reconstucted_patches = test_model(i)
     reconstructed_imgs[i*batch_size:(i+1)*batch_size,:,:,:] = reconstucted_patches

FSRCNN_Theano.rebuild_images(reconstructed_imgs,'/home/ubuntu/Data/Test_Subsamples_RGB_4/',patch_dim=17,dataset='test')


        

(512, 512, '\n')
(35, 35, '\n')
(288, 288, '\n')
(19, 19, '\n')
(256, 256, '\n')
(16, 16, '\n')
(280, 280, '\n')
(18, 18, '\n')
(344, 228, '\n')
(23, 14, '\n')
(480, 500, '\n')
(32, 34, '\n')
(576, 720, '\n')
(39, 50, '\n')
(512, 512, '\n')
(35, 35, '\n')
(288, 352, '\n')
(19, 23, '\n')
(361, 250, '\n')
(24, 16, '\n')
(276, 276, '\n')
(18, 18, '\n')
(362, 500, '\n')
(24, 34, '\n')
(288, 352, '\n')
(19, 23, '\n')
(512, 512, '\n')
(35, 35, '\n')
(512, 512, '\n')
(35, 35, '\n')
(512, 768, '\n')
(35, 53, '\n')
(512, 512, '\n')
(35, 35, '\n')
(656, 529, '\n')
(45, 36, '\n')
(391, 586, '\n')
(26, 40, '\n')


In [9]:
reconstructed_imgs = np.zeros(((n_valid_batches+1)*batch_size, 3, 17, 17))
for i in xrange(n_valid_batches):
     cost,MSE_per_pixel,psnr,reconstucted_patches = val_model(i)
     reconstructed_imgs[i*batch_size:(i+1)*batch_size,:,:,:] = reconstucted_patches

FSRCNN_Theano.rebuild_images(reconstructed_imgs,'/home/ubuntu/Data/Validation_Subsamples_RGB_4/',patch_dim=17,dataset='validate', place=True)
reconstructed_imgs = np.zeros((14851, 3, 17, 17))
for i in xrange(n_test_batches):
     cost,MSE_per_pixel,psnr,reconstucted_patches = test_model(i)
     reconstructed_imgs[i*batch_size:(i+1)*batch_size,:,:,:] = reconstucted_patches

FSRCNN_Theano.rebuild_images(reconstructed_imgs,'/home/ubuntu/Data/Test_Subsamples_RGB_4/',patch_dim=17,dataset='test',place=True)

(512, 512, '\n')
(35, 35, '\n')
(288, 288, '\n')
(19, 19, '\n')
(256, 256, '\n')
(16, 16, '\n')
(280, 280, '\n')
(18, 18, '\n')
(344, 228, '\n')
(23, 14, '\n')
(480, 500, '\n')
(32, 34, '\n')
(576, 720, '\n')
(39, 50, '\n')
(512, 512, '\n')
(35, 35, '\n')
(288, 352, '\n')
(19, 23, '\n')
(361, 250, '\n')
(24, 16, '\n')
(276, 276, '\n')
(18, 18, '\n')
(362, 500, '\n')
(24, 34, '\n')
(288, 352, '\n')
(19, 23, '\n')
(512, 512, '\n')
(35, 35, '\n')
(512, 512, '\n')
(35, 35, '\n')
(512, 768, '\n')
(35, 53, '\n')
(512, 512, '\n')
(35, 35, '\n')
(656, 529, '\n')
(45, 36, '\n')
(391, 586, '\n')
(26, 40, '\n')


10 epochs takes 1.4m with batchsize=20 
10 epochs takes .81m with batchsize=50
10 epochs takes 1.3m with batchsize=10
10 epochs takes .59m with batchsize=100 cost = 11404075.000000, mse/pixel = 8619.859375, pnsr = 8.786202

In [10]:
%autosave 300

Autosaving every 300 seconds


In [14]:
batch_size = 100
n_epochs = 100
lrs = [.0005]
for lr in lrs:
    print "\n\n ****************************** lr = " + str(lr) +"******************************************"
    learning_rate = lr

    n_train_batches = upsampled_x.shape[0]/batch_size
    n_valid_batches = up_val_x.shape[0]/batch_size
    n_test_batches = up_test_x.shape[0]/batch_size



    val_model,test_model = FSRCNN_Theano.train_FSRCNN(shared_x,shared_y,
                             shared_val_x,shared_val_y,
                             shared_test_x,shared_test_y,
                            n_train_batches, n_valid_batches, n_test_batches, 
                             n_epochs, batch_size,learning_rate,upsampling_factor=4)
reconstructed_imgs = np.zeros(((n_valid_batches+1)*batch_size, 3, 17, 17))
for i in xrange(n_valid_batches):
     cost,MSE_per_pixel,psnr,reconstucted_patches = val_model(i)
     reconstructed_imgs[i*batch_size:(i+1)*batch_size,:,:,:] = reconstucted_patches

FSRCNN_Theano.rebuild_images(reconstructed_imgs,'/home/ubuntu/Data/Validation_Subsamples_RGB_4/',patch_dim=17,dataset='validate_lr=5e4')
reconstructed_imgs = np.zeros((14851, 3, 17, 17))
for i in xrange(n_test_batches):
     cost,MSE_per_pixel,psnr,reconstucted_patches = test_model(i)
     reconstructed_imgs[i*batch_size:(i+1)*batch_size,:,:,:] = reconstucted_patches

FSRCNN_Theano.rebuild_images(reconstructed_imgs,'/home/ubuntu/Data/Test_Subsamples_RGB_4/',patch_dim=17,dataset='test_lr=5e4')

reconstructed_imgs = np.zeros(((n_valid_batches+1)*batch_size, 3, 17, 17))
for i in xrange(n_valid_batches):
     cost,MSE_per_pixel,psnr,reconstucted_patches = val_model(i)
     reconstructed_imgs[i*batch_size:(i+1)*batch_size,:,:,:] = reconstucted_patches

FSRCNN_Theano.rebuild_images(reconstructed_imgs,'/home/ubuntu/Data/Validation_Subsamples_RGB_4/',patch_dim=17,dataset='validate_lr=5e4', place=True)
reconstructed_imgs = np.zeros((14851, 3, 17, 17))
for i in xrange(n_test_batches):
     cost,MSE_per_pixel,psnr,reconstucted_patches = test_model(i)
     reconstructed_imgs[i*batch_size:(i+1)*batch_size,:,:,:] = reconstucted_patches

FSRCNN_Theano.rebuild_images(reconstructed_imgs,'/home/ubuntu/Data/Test_Subsamples_RGB_4/',patch_dim=17,dataset='test_lr=5e4',place=True)




 ****************************** lr = 0.0005******************************************
lovely....

filter_shape...: (64, 3, 9, 9)
image_shape...: (100, 3, 8, 8)
input shape....: Shape.0
lovely....

filter_shape...: (32, 64, 5, 5)
image_shape...: (100, 64, 8, 8)
input shape....: Shape.0
decon layer....

filter_shape...: (3, 32, 10, 10)
image_shape...: (100, 32, 8, 8)
input shape....: Shape.0
epoch 1, minibatch 220/220, validation cost 4955284.500000 mse/pixel: 5715.437988 pnsr: 11.090892
     epoch 1, minibatch 220/220, test cost of best model 5092713.500000 perpixel mse 5873.948242 and test pnsr 10.958442
epoch 2, minibatch 220/220, validation cost 2923540.750000 mse/pixel: 3372.019531 pnsr: 13.245140
     epoch 2, minibatch 220/220, test cost of best model 3319552.000000 perpixel mse 3828.779541 and test pnsr 12.793257
epoch 3, minibatch 220/220, validation cost 2336390.750000 mse/pixel: 2694.799072 pnsr: 14.212781
     epoch 3, minibatch 220/220, test cost of best model 2699663.5000

In [5]:
shared_x = theano.shared(np.asarray(upsampled_x,
                                       dtype=theano.config.floatX),
                         borrow=True)
shared_y = theano.shared(np.asarray(data_y,
                                       dtype=theano.config.floatX),
                         borrow=True)
shared_val_x = theano.shared(np.asarray(up_val_x,
                                       dtype=theano.config.floatX),
                         borrow=True)
shared_val_y = theano.shared(np.asarray(valid_y,
                                       dtype=theano.config.floatX),
                         borrow=True)
shared_test_x = theano.shared(np.asarray(up_test_x,
                                       dtype=theano.config.floatX),
                         borrow=True)
shared_test_y = theano.shared(np.asarray(test_y,
                                       dtype=theano.config.floatX),
                         borrow=True)
batch_size = 500
n_epochs = 100
lrs = [.0005]
for lr in lrs:
    print "\n\n ****************************** lr = " + str(lr) +"******************************************"
    learning_rate = lr

    n_train_batches = upsampled_x.shape[0]/batch_size
    n_valid_batches = up_val_x.shape[0]/batch_size
    n_test_batches = up_test_x.shape[0]/batch_size



    val_model,test_model = FSRCNN_Theano.train_FSRCNN(shared_x,shared_y,
                             shared_val_x,shared_val_y,
                             shared_test_x,shared_test_y,
                            n_train_batches, n_valid_batches, n_test_batches, 
                             n_epochs, batch_size,learning_rate,upsampling_factor=4)
reconstructed_imgs = np.zeros(((n_valid_batches+1)*batch_size, 3, 17, 17))
for i in xrange(n_valid_batches):
     cost,MSE_per_pixel,psnr,reconstucted_patches = val_model(i)
     reconstructed_imgs[i*batch_size:(i+1)*batch_size,:,:,:] = reconstucted_patches

FSRCNN_Theano.rebuild_images(reconstructed_imgs,'/home/ubuntu/Data/Validation_Subsamples_RGB_4/',patch_dim=17,dataset='validate_lr=5e4_batch=500')
reconstructed_imgs = np.zeros((14851, 3, 17, 17))
for i in xrange(n_test_batches):
     cost,MSE_per_pixel,psnr,reconstucted_patches = test_model(i)
     reconstructed_imgs[i*batch_size:(i+1)*batch_size,:,:,:] = reconstucted_patches

FSRCNN_Theano.rebuild_images(reconstructed_imgs,'/home/ubuntu/Data/Test_Subsamples_RGB_4/',patch_dim=17,dataset='test_lr=5e4_batch_500')

reconstructed_imgs = np.zeros(((n_valid_batches+1)*batch_size, 3, 17, 17))
for i in xrange(n_valid_batches):
     cost,MSE_per_pixel,psnr,reconstucted_patches = val_model(i)
     reconstructed_imgs[i*batch_size:(i+1)*batch_size,:,:,:] = reconstucted_patches

FSRCNN_Theano.rebuild_images(reconstructed_imgs,'/home/ubuntu/Data/Validation_Subsamples_RGB_4/',patch_dim=17,dataset='validate_lr=5e4_batch=500', place=True)
reconstructed_imgs = np.zeros((14851, 3, 17, 17))
for i in xrange(n_test_batches):
     cost,MSE_per_pixel,psnr,reconstucted_patches = test_model(i)
     reconstructed_imgs[i*batch_size:(i+1)*batch_size,:,:,:] = reconstucted_patches

FSRCNN_Theano.rebuild_images(reconstructed_imgs,'/home/ubuntu/Data/Test_Subsamples_RGB_4/',patch_dim=17,dataset='test_lr=5e4_batch=500',place=True)




 ****************************** lr = 0.0005******************************************
lovely....

filter_shape...: (64, 3, 9, 9)
image_shape...: (500, 3, 8, 8)
input shape....: Shape.0
lovely....

filter_shape...: (32, 64, 5, 5)
image_shape...: (500, 64, 8, 8)
input shape....: Shape.0
decon layer....

filter_shape...: (3, 32, 10, 10)
image_shape...: (500, 32, 8, 8)
input shape....: Shape.0
training @ iter = 0	cost = 35665.5117188	mse/pixel = 41.1366920471	 pnsr = 31.9885082245
epoch 1, minibatch 44/44, validation cost 12238.873047 mse/pixel: 14.116347 pnsr: 37.206314
     epoch 1, minibatch 44/44, test cost of best model 12823.865234 perpixel mse 14.791079 and test pnsr 36.898827
epoch 2, minibatch 44/44, validation cost 8567.228516 mse/pixel: 9.881462 pnsr: 38.656013
     epoch 2, minibatch 44/44, test cost of best model 9353.253906 perpixel mse 10.788068 and test pnsr 38.280449
training @ iter = 100	cost = 8581.75390625	mse/pixel = 9.89821720123	 pnsr = 38.1752319336
epoch 3, minib

In [6]:
shared_x = theano.shared(np.asarray(upsampled_x,
                                       dtype=theano.config.floatX),
                         borrow=True)
shared_y = theano.shared(np.asarray(data_y,
                                       dtype=theano.config.floatX),
                         borrow=True)
shared_val_x = theano.shared(np.asarray(up_val_x,
                                       dtype=theano.config.floatX),
                         borrow=True)
shared_val_y = theano.shared(np.asarray(valid_y,
                                       dtype=theano.config.floatX),
                         borrow=True)
shared_test_x = theano.shared(np.asarray(up_test_x,
                                       dtype=theano.config.floatX),
                         borrow=True)
shared_test_y = theano.shared(np.asarray(test_y,
                                       dtype=theano.config.floatX),
                         borrow=True)
batch_size = 50
n_epochs = 100
lrs = [.0005]
for lr in lrs:
    print "\n\n ****************************** lr = " + str(lr) +"******************************************"
    learning_rate = lr

    n_train_batches = upsampled_x.shape[0]/batch_size
    n_valid_batches = up_val_x.shape[0]/batch_size
    n_test_batches = up_test_x.shape[0]/batch_size



    val_model,test_model = FSRCNN_Theano.train_FSRCNN(shared_x,shared_y,
                             shared_val_x,shared_val_y,
                             shared_test_x,shared_test_y,
                            n_train_batches, n_valid_batches, n_test_batches, 
                             n_epochs, batch_size,learning_rate,upsampling_factor=4)
reconstructed_imgs = np.zeros(((n_valid_batches+1)*batch_size, 3, 17, 17))
for i in xrange(n_valid_batches):
     cost,MSE_per_pixel,psnr,reconstucted_patches = val_model(i)
     reconstructed_imgs[i*batch_size:(i+1)*batch_size,:,:,:] = reconstucted_patches

FSRCNN_Theano.rebuild_images(reconstructed_imgs,'/home/ubuntu/Data/Validation_Subsamples_RGB_4/',patch_dim=17,dataset='validate_lr=5e4_batch=500')
reconstructed_imgs = np.zeros((14851, 3, 17, 17))
for i in xrange(n_test_batches):
     cost,MSE_per_pixel,psnr,reconstucted_patches = test_model(i)
     reconstructed_imgs[i*batch_size:(i+1)*batch_size,:,:,:] = reconstucted_patches

FSRCNN_Theano.rebuild_images(reconstructed_imgs,'/home/ubuntu/Data/Test_Subsamples_RGB_4/',patch_dim=17,dataset='test_lr=5e4_batch_500')

reconstructed_imgs = np.zeros(((n_valid_batches+1)*batch_size, 3, 17, 17))
for i in xrange(n_valid_batches):
     cost,MSE_per_pixel,psnr,reconstucted_patches = val_model(i)
     reconstructed_imgs[i*batch_size:(i+1)*batch_size,:,:,:] = reconstucted_patches

FSRCNN_Theano.rebuild_images(reconstructed_imgs,'/home/ubuntu/Data/Validation_Subsamples_RGB_4/',patch_dim=17,dataset='validate_lr=5e4_batch=50', place=True)
reconstructed_imgs = np.zeros((14851, 3, 17, 17))
for i in xrange(n_test_batches):
     cost,MSE_per_pixel,psnr,reconstucted_patches = test_model(i)
     reconstructed_imgs[i*batch_size:(i+1)*batch_size,:,:,:] = reconstucted_patches

FSRCNN_Theano.rebuild_images(reconstructed_imgs,'/home/ubuntu/Data/Test_Subsamples_RGB_4/',patch_dim=17,dataset='test_lr=5e4_batch=50',place=True)




 ****************************** lr = 0.0005******************************************
lovely....

filter_shape...: (64, 3, 9, 9)
image_shape...: (50, 3, 8, 8)
input shape....: Shape.0
lovely....

filter_shape...: (32, 64, 5, 5)
image_shape...: (50, 64, 8, 8)
input shape....: Shape.0
decon layer....

filter_shape...: (3, 32, 10, 10)
image_shape...: (50, 32, 8, 8)
input shape....: Shape.0
training @ iter = 0	cost = 49953.8007812	mse/pixel = 57.6168441772	 pnsr = 30.525308609
training @ iter = 100	cost = 9480.41210938	mse/pixel = 10.9347314835	 pnsr = 37.7427215576
training @ iter = 200	cost = 3995.08813477	mse/pixel = 4.60794496536	 pnsr = 41.4957275391
training @ iter = 300	cost = 4732.83300781	mse/pixel = 5.45886182785	 pnsr = 40.7597808838
training @ iter = 400	cost = 3787.21508789	mse/pixel = 4.36818361282	 pnsr = 41.7277946472
epoch 1, minibatch 441/441, validation cost 3776.111572 mse/pixel: 4.355378 pnsr: 42.377975
     epoch 1, minibatch 441/441, test cost of best model 4403.46

/mnt/anaconda2/lib/python2.7/site-packages/matplotlib/pyplot.py:516: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


In [7]:
shared_x = theano.shared(np.asarray(upsampled_x,
                                       dtype=theano.config.floatX),
                         borrow=True)
shared_y = theano.shared(np.asarray(data_y,
                                       dtype=theano.config.floatX),
                         borrow=True)
shared_val_x = theano.shared(np.asarray(up_val_x,
                                       dtype=theano.config.floatX),
                         borrow=True)
shared_val_y = theano.shared(np.asarray(valid_y,
                                       dtype=theano.config.floatX),
                         borrow=True)
shared_test_x = theano.shared(np.asarray(up_test_x,
                                       dtype=theano.config.floatX),
                         borrow=True)
shared_test_y = theano.shared(np.asarray(test_y,
                                       dtype=theano.config.floatX),
                         borrow=True)
batch_size = 50
n_epochs = 100
lrs = [.0005]
for lr in lrs:
    print "\n\n ****************************** lr = " + str(lr) +"******************************************"
    learning_rate = lr

    n_train_batches = upsampled_x.shape[0]/batch_size
    n_valid_batches = up_val_x.shape[0]/batch_size
    n_test_batches = up_test_x.shape[0]/batch_size



    val_model,test_model = FSRCNN_Theano.train_FSRCNN3(shared_x,shared_y,
                             shared_val_x,shared_val_y,
                             shared_test_x,shared_test_y,
                            n_train_batches, n_valid_batches, n_test_batches, 
                             n_epochs, batch_size,learning_rate,upsampling_factor=4)
reconstructed_imgs = np.zeros(((n_valid_batches+1)*batch_size, 3, 16, 16))
for i in xrange(n_valid_batches):
     cost,MSE_per_pixel,psnr,reconstucted_patches = val_model(i)
     reconstructed_imgs[i*batch_size:(i+1)*batch_size,:,:,:] = reconstucted_patches

FSRCNN_Theano.rebuild_images(reconstructed_imgs,'/home/ubuntu/Data/Validation_Subsamples_RGB_4/',patch_dim=16,dataset='validate_lr=5e4_batch=50_model3')
reconstructed_imgs = np.zeros((14851, 3, 16, 16))
for i in xrange(n_test_batches):
     cost,MSE_per_pixel,psnr,reconstucted_patches = test_model(i)
     reconstructed_imgs[i*batch_size:(i+1)*batch_size,:,:,:] = reconstucted_patches

FSRCNN_Theano.rebuild_images(reconstructed_imgs,'/home/ubuntu/Data/Test_Subsamples_RGB_4/',patch_dim=16,dataset='test_lr=5e4_batch_50_model3')

reconstructed_imgs = np.zeros(((n_valid_batches+1)*batch_size, 3, 16, 16))
for i in xrange(n_valid_batches):
     cost,MSE_per_pixel,psnr,reconstucted_patches = val_model(i)
     reconstructed_imgs[i*batch_size:(i+1)*batch_size,:,:,:] = reconstucted_patches

FSRCNN_Theano.rebuild_images(reconstructed_imgs,'/home/ubuntu/Data/Validation_Subsamples_RGB_4/',patch_dim=16,dataset='validate_lr=5e4_batch=50_model3', place=True)
reconstructed_imgs = np.zeros((14851, 3, 16, 16))
for i in xrange(n_test_batches):
     cost,MSE_per_pixel,psnr,reconstucted_patches = test_model(i)
     reconstructed_imgs[i*batch_size:(i+1)*batch_size,:,:,:] = reconstucted_patches

FSRCNN_Theano.rebuild_images(reconstructed_imgs,'/home/ubuntu/Data/Test_Subsamples_RGB_4/',patch_dim=16,dataset='test_lr=5e4_batch=50_model3',place=True)



 ****************************** lr = 0.0005******************************************
lovely....

filter_shape...: (64, 3, 9, 9)
image_shape...: (50, 3, 8, 8)
input shape....: Shape.0
lovely....

filter_shape...: (32, 64, 5, 5)
image_shape...: (50, 64, 8, 8)
input shape....: Shape.0
decon layer....

filter_shape...: (3, 32, 9, 9)
image_shape...: (50, 32, 8, 8)
input shape....: Shape.0
training @ iter = 0	cost = 60621.9492188	mse/pixel = 78.9348297119	 pnsr = 29.158115387
training @ iter = 100	cost = 13509.5839844	mse/pixel = 17.5906028748	 pnsr = 35.6779937744
training @ iter = 200	cost = 9165.96582031	mse/pixel = 11.9348516464	 pnsr = 37.3626327515
training @ iter = 300	cost = 3908.55175781	mse/pixel = 5.08926010132	 pnsr = 41.0642547607
training @ iter = 400	cost = 3797.87353516	mse/pixel = 4.94514751434	 pnsr = 41.1890106201
epoch 1, minibatch 441/441, validation cost 3683.568604 mse/pixel: 4.796313 pnsr: 41.923969
     epoch 1, minibatch 441/441, test cost of best model 4415.0185

In [4]:
shared_x = theano.shared(np.asarray(upsampled_x,
                                       dtype=theano.config.floatX),
                         borrow=True)
shared_y = theano.shared(np.asarray(data_y,
                                       dtype=theano.config.floatX),
                         borrow=True)
shared_val_x = theano.shared(np.asarray(up_val_x,
                                       dtype=theano.config.floatX),
                         borrow=True)
shared_val_y = theano.shared(np.asarray(valid_y,
                                       dtype=theano.config.floatX),
                         borrow=True)
shared_test_x = theano.shared(np.asarray(up_test_x,
                                       dtype=theano.config.floatX),
                         borrow=True)
shared_test_y = theano.shared(np.asarray(test_y,
                                       dtype=theano.config.floatX),
                         borrow=True)
batch_size = 25
n_epochs = 100
lrs = [.0005]
for lr in lrs:
    print "\n\n ****************************** lr = " + str(lr) +"******************************************"
    learning_rate = lr

    n_train_batches = upsampled_x.shape[0]/batch_size
    n_valid_batches = up_val_x.shape[0]/batch_size
    n_test_batches = up_test_x.shape[0]/batch_size



    val_model,test_model = FSRCNN_Theano.train_FSRCNN3(shared_x,shared_y,
                             shared_val_x,shared_val_y,
                             shared_test_x,shared_test_y,
                            n_train_batches, n_valid_batches, n_test_batches, 
                             n_epochs, batch_size,learning_rate,upsampling_factor=4)
reconstructed_imgs = np.zeros(((n_valid_batches+1)*batch_size, 3, 16, 16))
for i in xrange(n_valid_batches):
     cost,MSE_per_pixel,psnr,reconstucted_patches = val_model(i)
     reconstructed_imgs[i*batch_size:(i+1)*batch_size,:,:,:] = reconstucted_patches

FSRCNN_Theano.rebuild_images(reconstructed_imgs,'/home/ubuntu/Data/Validation_Subsamples_RGB_4/',patch_dim=16,dataset='validate_lr=5e4_batch=25_model3')
reconstructed_imgs = np.zeros((14851, 3, 16, 16))
for i in xrange(n_test_batches):
     cost,MSE_per_pixel,psnr,reconstucted_patches = test_model(i)
     reconstructed_imgs[i*batch_size:(i+1)*batch_size,:,:,:] = reconstucted_patches

FSRCNN_Theano.rebuild_images(reconstructed_imgs,'/home/ubuntu/Data/Test_Subsamples_RGB_4/',patch_dim=16,dataset='test_lr=5e4_batch_25_model3')

reconstructed_imgs = np.zeros(((n_valid_batches+1)*batch_size, 3, 16, 16))
for i in xrange(n_valid_batches):
     cost,MSE_per_pixel,psnr,reconstucted_patches = val_model(i)
     reconstructed_imgs[i*batch_size:(i+1)*batch_size,:,:,:] = reconstucted_patches

FSRCNN_Theano.rebuild_images(reconstructed_imgs,'/home/ubuntu/Data/Validation_Subsamples_RGB_4/',patch_dim=16,dataset='validate_lr=5e4_batch=25_model3', place=True)
reconstructed_imgs = np.zeros((14851, 3, 16, 16))
for i in xrange(n_test_batches):
     cost,MSE_per_pixel,psnr,reconstucted_patches = test_model(i)
     reconstructed_imgs[i*batch_size:(i+1)*batch_size,:,:,:] = reconstucted_patches

FSRCNN_Theano.rebuild_images(reconstructed_imgs,'/home/ubuntu/Data/Test_Subsamples_RGB_4/',patch_dim=16,dataset='test_lr=5e4_batch=25_model3',place=True)



 ****************************** lr = 0.0005******************************************
lovely....

filter_shape...: (64, 3, 9, 9)
image_shape...: (25, 3, 8, 8)
input shape....: Shape.0
lovely....

filter_shape...: (32, 64, 5, 5)
image_shape...: (25, 64, 8, 8)
input shape....: Shape.0
decon layer....

filter_shape...: (3, 32, 9, 9)
image_shape...: (25, 32, 8, 8)
input shape....: Shape.0
training @ iter = 0	cost = 53047.1640625	mse/pixel = 69.0718307495	 pnsr = 29.7377929688
training @ iter = 100	cost = 17049.5957031	mse/pixel = 22.1999931335	 pnsr = 34.6672744751
training @ iter = 200	cost = 1910.13256836	mse/pixel = 2.48715186119	 pnsr = 44.1737785339
training @ iter = 300	cost = 1641.56103516	mse/pixel = 2.13744926453	 pnsr = 44.8318443298
training @ iter = 400	cost = 5444.76611328	mse/pixel = 7.08953905106	 pnsr = 39.624622345
training @ iter = 500	cost = 1352.5645752	mse/pixel = 1.76115179062	 pnsr = 45.672832489
training @ iter = 600	cost = 2573.82104492	mse/pixel = 3.35132956505	

/mnt/anaconda2/lib/python2.7/site-packages/matplotlib/pyplot.py:516: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)
